In [ ]:
import json
import sys
from pathlib import Path
sys.path.append(str(Path("../../../../")))
from research_szz_cregit.models.commit_map import CommitMap
from research_szz_cregit.repositories.commit_map.impl import CommitMapRepository
import copy

BASE_DIR = Path().resolve().parent.parent.parent.parent
DATABASE_DIR = BASE_DIR / "dataset" / "databases" / "developer-informed-oracle" /"1token"

with open(
    BASE_DIR
    / "dataset"
    / "pyszz_v2"
    / "json-output-raw"
    / "output"
    / "developer-informed-oracle"
    / "dio_bic_conf_1token.json",
    "r",
) as f:
    buginducing_commits = json.load(f)


res_cregit = []

for buginducing_commit in buginducing_commits:
    print(buginducing_commit["repo_name"])

    owner, project = buginducing_commit["repo_name"].split("/")

    commit_map_repository = CommitMapRepository(
        DATABASE_DIR / Path(f"{owner}/{project}/{project}-cregit.db")
    )

    commit_map = commit_map_repository.get_all()
    # cregit to original
    hashmaped_commit_map = CommitMap.get_cid_hashmap(commit_map)
    # original to cregit
    hashmaped_commit_map_from_original = CommitMap.get_cid_hashmap_from_original_cid(
        commit_map
    )

    try:
        buginducing_commit["fix_commit_hash"] = hashmaped_commit_map[
            buginducing_commit["fix_commit_hash"]
        ].original_cid
        buginducing_commit["bug_commit_hash"] = [
            hashmaped_commit_map[commit].original_cid
            for commit in buginducing_commit["bug_commit_hash"]
        ]
        inducing_commit_hash = []
        for commit in buginducing_commit["inducing_commit_hash"]:
            recoverd = copy.deepcopy(commit)
            recoverd["commit_hash"] = hashmaped_commit_map[
                recoverd["commit_hash"]
            ].original_cid
            inducing_commit_hash.append(recoverd)
        buginducing_commit["inducing_commit_hash"] = inducing_commit_hash

        res_cregit.append(buginducing_commit)
    except KeyError:
        print(
            f"KeyError on {buginducing_commit['repo_name']} {buginducing_commit['fix_commit_hash']}"
        )

with open(BASE_DIR
    / "dataset"
    / "pyszz_v2"
    / "json-output-raw"
    / "rq1"
    / "dio_bic_conf_5-gram_no_filter.json", "w") as f:
    json.dump(res_cregit, f, indent=4)
